In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df = spark.read.format("parquet").load("abfss://bronze@eklavyadatabricks.dfs.core.windows.net/customers")

In [0]:

df = df.drop("_rescued_data")
df = df.withColumn("domain",split(col("email"),"@")[1])
display(df)

In [0]:
df_2 = df.groupBy("domain").agg(count("customer_id").alias("total_customers")).orderBy(col("total_customers").desc())
#alternate_way to sort
df_3 = df.groupBy("domain").agg(count("customer_id").alias("total_customers")).sort("total_customers",ascending=False)
display(df_3)

In [0]:

import time

df_gmail = df.filter(col("domain")=="gmail.com")
display(df_gmail)
time.sleep(5)

df_hotmail = df.filter(col("domain")=="hotmail.com")
display(df_hotmail)
time.sleep(5)

df_yahoo = df.filter(col("domain")=="yahoo.com")
display(df_yahoo)
time.sleep(5)

In [0]:
df = df.withColumn("full_name",concat(col("first_name"),lit(" "),col("last_name")))
df=df.drop(col("first_name"),col("last_name"))
display(df)

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@eklavyadatabricks.dfs.core.windows.net/customers")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.customers
USING DELTA
LOCATION 'abfss://silver@eklavyadatabricks.dfs.core.windows.net/customers'